**数据描述**

- 数据来源：https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data
- 数据说明：
    - 第 1 列为检索的 id
    - 2 - 10 列为医学特征
    - 第 11 列为标签，2->良性  4->恶性
    - 共有 16 个缺失值，用 ? 标出
    - 每个特征都被量化到 1 - 10 之间的数值

In [1]:
import pandas as pd
import numpy as np

数据文件是一个没有 Header 的 CSV 文件
```
1000025,5,1,1,1,2,1,3,1,1,2
1002945,5,4,4,5,7,10,3,2,1,2
1015425,3,1,1,1,2,2,3,1,1,2
1016277,6,8,8,1,3,4,3,7,1,2
1017023,4,1,1,3,2,1,3,1,1,2
1017122,8,10,10,8,7,10,9,7,1,4
1018099,1,1,1,1,2,10,3,1,1,2
1018561,2,1,2,1,2,1,3,1,1,2
1033078,2,1,1,1,2,1,1,1,5,2
...
...
```

In [2]:
# 为数据的每一列指定名字
column_names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Size', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class']

In [3]:
# 读入数据，并为每一列指定之前定义好的列名
data = pd.read_csv('breast-cancer-wisconsin.data', names=column_names)

In [4]:
# 将缺失值替换为'标准缺失值'
data = data.replace(to_replace='?', value=np.nan)
data.shape

(699, 11)

In [5]:
# 将带有缺失值的数据丢掉，只要包含 NaN 就丢弃掉该行
data = data.dropna(how='any')

In [6]:
data.shape

(683, 11)

由于数据集没有专门的用于测试用的数据，所以我们需要将原始数据分为 train(75%) 和 test(25%) 两部分

In [7]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[column_names[1:10]], data[column_names[10]], test_size=0.25, random_state=33)

/Users/willing/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
# 查验分割后数据的分布
y_train.value_counts()

2    344
4    168
Name: Class, dtype: int64

In [9]:
y_test.value_counts()

2    100
4     71
Name: Class, dtype: int64

分别用 1. 逻辑斯特回归 2. 随机梯度参数估计

In [10]:
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [11]:
# 标准化数据，保证每个维度的特征数据方差为 1，均值为 0。使得预测结果不会被某些维度过大的特征主导
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [12]:
X_train

array([[-1.19196677, -0.68788787, -0.68788787, ..., -0.55666054,
        -0.58626819, -0.34343195],
       [-1.19196677,  0.30608745,  0.30608745, ...,  0.68611648,
         1.09441105, -0.34343195],
       [ 0.24231522, -0.02523765, -0.02523765, ..., -0.97091955,
        -0.58626819, -0.34343195],
       ..., 
       [ 0.24231522, -0.68788787, -0.68788787, ..., -0.97091955,
        -0.58626819, -0.34343195],
       [-1.19196677, -0.68788787, -0.68788787, ..., -0.55666054,
        -0.58626819, -0.34343195],
       [ 0.95945621, -0.02523765, -0.02523765, ...,  0.68611648,
         0.42213936,  1.42674078]])

In [13]:
lr = LogisticRegression()
sgdc = SGDClassifier()

lr.fit(X_train, y_train)
lr_y_predict = lr.predict(X_test)

sgdc.fit(X_train, y_train)
sgdc_y_predict = sgdc.predict(X_test)

我们的样本中只有两类

- 阳性->恶性肿瘤
- 阴性->良性肿瘤

在我们的预测结果中，其中我们

- 预测正确的阳性，称为真阳性(True Positive)
- 预测错误的阳性，称为假阳性(False Positive)
- 预测正确的阴性，称为真阴性(True Negative)
- 预测错误的阴性，称为假阴性(False Negative)

$$
Accuracy(准确性) = \frac{True\ Positive + True\ Negative}{True\ Positive + False\ Positive + True\ Negative + False\ Negative}
$$

$$
Precision(精度）= \frac{True\ Positve}{True\ Positive + False\ Positive}
$$

$$
Recall(召回) = \frac{True\ Positive}{True\ Positive + False\ Negative}
$$

在我们的这个例子中，召回率就显得特别重要，因为我们不希望将一个有病的人误诊为无病，从而耽误了最佳治疗时间

为了综合考量**精度**和**召回**，我们引入了**调和平均数**，即**F1 指标**

$$
F1\ measure = \frac{2}{\frac{1}{Precision} + \frac{1}{Recall}}
$$

对于精度和召回接近的模型给予更高的分数，因为那些召回率和精确率差距过大的学习模型，往往没有足够的使用价值。

MathJax 公式语法：
$ x=\frac{-b\pm\sqrt{b^2-4ac}}{2a} $

In [14]:
# 我们将在 准确性、召回率、精确率、F1 指标 4 个方面评价我们的模型

from sklearn.metrics import classification_report

print 'Accuracy of LogisticRegression Classifier:', lr.score(X_test, y_test)
print classification_report(y_test, lr_y_predict, target_names=['Benign', 'Malignant'])

Accuracy of LogisticRegression Classifier: 0.988304093567
             precision    recall  f1-score   support

     Benign       0.99      0.99      0.99       100
  Malignant       0.99      0.99      0.99        71

avg / total       0.99      0.99      0.99       171



In [15]:
print 'Accuracy of SGDClassifier:', sgdc.score(X_test, y_test)
print classification_report(y_test, sgdc_y_predict, target_names=['Benign', 'Malignant'])

Accuracy of SGDClassifier: 0.900584795322
             precision    recall  f1-score   support

     Benign       0.87      0.97      0.92       100
  Malignant       0.95      0.80      0.87        71

avg / total       0.91      0.90      0.90       171



在使用 SGDClassifier fit 的过程中，每一次都有差别，应该是源于，初始化时的 weight 梯度不同导致的